In [3]:
import hashlib
import os

# Compute the greatest common divisor of two numbers using Euclid's algorithm.
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Compute the modular multiplicative inverse of 'a' under modulo 'm'.
def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1

# Check if a given number is prime.
def is_prime(num):
    if num <= 1:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

# Generate an RSA key pair using two prime numbers 'p' and 'q'.
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    n = p * q
    phi = (p-1) * (q-1)
    e = 2
    while e < phi:
        if gcd(e, phi) == 1:
            break
        e += 1
    d = modinv(e, phi)
    return ((e, n), (d, n))

# Encrypt a plaintext using the RSA public key.
def encrypt(pk, plaintext):
    e, n = pk
    return [pow(ord(char), e, n) for char in plaintext]

# Decrypt a ciphertext using the RSA private key.
def decrypt(pk, ciphertext):
    d, n = pk
    return ''.join([chr(pow(char, d, n)) for char in ciphertext])

# Sign a message using the RSA private key.
# The signature ensures message authenticity and integrity.
def sign(private_key, message):
    # Convert the message into a SHA-256 hash.
    message_hash = int(hashlib.sha256(message.encode()).hexdigest(), 16)
    d, n = private_key
    signature = pow(message_hash % n, d, n)
    return signature

# Verify the authenticity of a message using its signature and the RSA public key.
def verify(public_key, message, signature):
    # Convert the message into a SHA-256 hash.
    message_hash = int(hashlib.sha256(message.encode()).hexdigest(), 16)
    e, n = public_key
    decrypted_hash = pow(signature, e, n)
    return message_hash % n == decrypted_hash

# Main function for demonstrating the RSA encryption, decryption, and digital signature.
if __name__ == '__main__':
    print("RSA Encryption/Decryption")

    # Ensure the first prime number is valid.
    while True:
        p = int(input("Enter a prime number (e.g., 17, 19, 23): "))
        if is_prime(p):
            break
        else:
            print("Please enter a valid prime number.")

    # Ensure the second prime number is valid and different from the first.
    while True:
        q = int(input("Enter another prime number (different from the first): "))
        if is_prime(q) and p != q:
            break
        else:
            print("Please enter a valid prime number different from the first.")

    # Generate RSA keys.
    public, private = generate_keypair(p, q)
    print("\nPublic key:", public)
    print("Private key:", private)

    # Decide the source of the message: direct input or from a file.
    choice = input("\nWant to input the message directly or from a .txt file? (m/f): ").lower()
    if choice == 'm':
        message = input("Enter a message to encrypt (small data): ")
    elif choice == 'f':
        filename = input("Enter the path to the .txt file: ")
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                message = f.read()
        else:
            print("File not found!")
            exit()

    # Encryption and Decryption demonstration.
    encrypted_msg = encrypt(public, message)
    print("\nEncrypted message:", ''.join(map(lambda x: str(x), encrypted_msg)))
    decrypted_msg = decrypt(private, encrypted_msg)
    print("Decrypted message:", decrypted_msg)

    # Digital Signature demonstration.
    signature = sign(private, message)
    print("\nDigital Signature:", signature)
    is_verified = verify(public, message, signature)
    if is_verified:
        print("The signature is valid!")
    else:
        print("The signature is invalid!")


RSA Encryption/Decryption
Enter a prime number (e.g., 17, 19, 23): 109
Enter another prime number (different from the first): 103

Public key: (5, 11227)
Private key: (8813, 11227)

Want to input the message directly or from a .txt file? (m/f): m
Enter a message to encrypt (small data): hi, 102 , # mukand

Encrypted message: 87566933282181563929720376828156282181561969815665401812792510043218110057
Decrypted message: hi, 102 , # mukand

Digital Signature: 3107
The signature is valid!
